### Определение координат границы между между Россией и Украиной

In [ ]:
#Библиотека для работы с многомерными массивами
!pip install numpy
#Библиотека для обработки и анализа данных
!pip install pandas
#Библиотека для библиотека для визуализации данных
!pip install plotly
#Библиотека для упрощения работы с геопространственными данными
!pip install geopandas 

In [1]:
# Импорт библиотек для работы с данными
import json           #Библиотека для работы с ".json"-файлами
import pandas as pd   #Библиотека для обработки и анализа данных
import numpy as np    #Библиотека для работы с многомерными массивами

In [2]:
# Считываем файл с координатами всех регионов
with open('data//gadm41_RUS_1.json', encoding = 'utf-8') as js:
    dict_coordin_border = json.load(js)

In [3]:
for regions in dict_coordin_border['features']:
    print(regions['properties']['NL_NAME_1'])

РеспубликаАдыгея
Алтайскийкрай
Амурскаяобласть
Архангельскаяобласть
Астраханскаяобласть
РеспубликаБашкортостан
Белгородскаяобласть
Брянскаяобласть
РеспубликаБурятия
РеспубликаЧечено-Ингушская
Челябинскаяобласть
ЧукотскийАОк
ЧувашскаяРеспублика
Санкт-Петербург(горсовет)
РеспубликаДагестан
РеспубликаАлтай
Респу́бликаИнгуше́тия
Иркутскаяобласть
Ивановскаяобласть
Кабардино-БалкарскаяРеспублика
Калининградскаяобласть
РеспубликаКалмыкия
Калужскаяобласть
Камчатскаякрай
Карачаево-ЧеркессияРеспублика
РеспубликаКарелия
Кемеровскаяобласть
Хабаровскийкрай
РеспубликаХакасия
Ханты-МансийскийАОк
Кировскаяобласть
РеспубликаКоми
Костромскаяобласть
Краснодарскийкрай
Красноярскийкрай
Курганскаяобласть
Курскаяобласть
Ленинградскаяобласть
Липецкаяобласть
Магадан|Магаданскаяобласть
РеспубликаМарийЭл
РеспубликаМордовия
NA
Московскаяобласть
Мурманскаяобласть
НенецкийАОк
Нижегородскаяобласть
РеспубликаСевернаяОсетия-Алани
Новгородскаяобласть
Новосибирскаяобласть
Омскаяобласть
Орловскаяобласть
Оренбургскаяоблас

### Нахождение координат точек границ соседних с Украиной регионов

In [ ]:
"""
Выбираем субъекты федерации, которые граничат с Украиной:
1. Воронежская область
2. Брянская область
3. Курская область
4. Ростовская область
5. Белгородская область
"""

#Создаем dataframe для записи данных
df_coord_reg = pd.DataFrame()
sequence_number = 0

for regions in dict_coordin_border['features']:
    #Ставим условия для поля названия субъектов
    if regions['properties']['NL_NAME_1'] in ['Воронежскаяобласть','Брянскаяобласть','Курскаяобласть','Ростовскаяобласть', 'Белгородскаяобласть']:
        for list_coordin_lv_1 in regions['geometry']['coordinates']:
            for list_coordin_lv_2 in list_coordin_lv_1:
                for list_coordin_finish_lvl in list_coordin_lv_2:
                    #Заполняем df: Название региона, координаты точки границы, порядковый номер записи, цвет региона
                    if regions['properties']['NL_NAME_1'] == 'Воронежскаяобласть': color = 'purple'
                    if regions['properties']['NL_NAME_1'] == 'Брянскаяобласть': color = 'white'
                    if regions['properties']['NL_NAME_1'] == 'Курскаяобласть': color = 'blue'
                    if regions['properties']['NL_NAME_1'] == 'Ростовскаяобласть': color = 'yellow'
                    if regions['properties']['NL_NAME_1'] == 'Белгородскаяобласть': color = 'red'
                        
                    df_coord_reg = df_coord_reg.append({'region': regions['properties']['NL_NAME_1'], #Название региона
                                                        'lon': list_coordin_finish_lvl[0], #Долгота точки границы
                                                        'lat': list_coordin_finish_lvl[1], #Широта точки границы
                                                        'sequence_number': str(sequence_number), #Порядковый номер записи
                                                        'color': color}, #Цвет региона 
                                                       ignore_index = True)
                    sequence_number += 1

In [5]:
#Посмотрим на получившийся df
df_coord_reg

,region,lon,lat,sequence_number,color
0,Белгородскаяобласть,39.2482,50.1158,0,red
1,Белгородскаяобласть,39.2858,50.0916,1,red
2,Белгородскаяобласть,39.3360,50.0704,2,red
3,Белгородскаяобласть,39.3649,50.0454,3,red
4,Белгородскаяобласть,39.3743,49.9940,4,red
...,...,...,...,...,...
3158,Воронежскаяобласть,39.2647,49.8441,3158,purple
3159,Воронежскаяобласть,39.2916,49.8500,3159,purple
3160,Воронежскаяобласть,39.3008,49.8961,3160,purple
3161,Воронежскаяобласть,39.3157,49.9306,3161,purple


### Визуализируем полученные данные на карте Европы и выберим нужные точки границы

In [10]:
#Импортируем библиотеки для визуализации данных
import plotly.graph_objs as go
from plotly.graph_objs import Scatter, Figure, Layout

In [11]:
#Визуализируем на карте точки с координатами для проверки и дальнейшего анализа

fig = go.Figure(data=go.Scattergeo( #Scattergeo - данные, визуализируемые в виде точек географической карте
                  lon = df_coord_reg['lon'],                                               #Долгота точки
                  lat = df_coord_reg['lat'],                                               #Широта точки
                  mode = 'markers',                                                        #Вид точки
                  marker_color  = df_coord_reg['color'],                                   #Цвет точки
                  text = df_coord_reg['region'] + ' ' + df_coord_reg['sequence_number']    #Текст при наведении на точку
                )
            )

fig.update_layout(
        title = 'Субъекты РФ граничащие с Украиной ',      #Задаем название карты
        geo = dict(
            scope='europe',                                #Шаблон карты 
            landcolor = "green",                           #Цвет для стран
            countrycolor = "black",                        #Цвет границ между странами
        ),
    )

In [12]:
#Исходя из карты, выбираем следующие срезы df и записываем их в новую переменную 
df_coord_border = pd.concat([df_coord_reg[11:411], 
                             df_coord_reg[1278:1459], 
                             df_coord_reg[974:1226], 
                             df_coord_reg[3084:3157],
                             df_coord_reg[2004:2413]])

In [13]:
df_coord_border

,region,lon,lat,sequence_number,color
11,Белгородскаяобласть,39.2300,49.8384,11,red
12,Белгородскаяобласть,39.2218,49.8444,12,red
13,Белгородскаяобласть,39.2224,49.8498,13,red
14,Белгородскаяобласть,39.2136,49.8503,14,red
15,Белгородскаяобласть,39.2142,49.8557,15,red
...,...,...,...,...,...
2408,Ростовскаяобласть,40.1633,49.4581,2408,yellow
2409,Ростовскаяобласть,40.1850,49.4758,2409,yellow
2410,Ростовскаяобласть,40.1978,49.4909,2410,yellow
2411,Ростовскаяобласть,40.2174,49.5070,2411,yellow


### Визуально проверяем полученный dataframe 

In [14]:
fig = go.Figure(data=go.Scattergeo(lon = df_coord_border['lon'], 
                                   lat = df_coord_border['lat'], 
                                   mode = 'markers',
                                   marker_color  = df_coord_border['color'],
                                   text = df_coord_border['region'] + ' ' + df_coord_border['sequence_number']))
fig.update_layout(
        title = 'Граница между Россией и Украиной',
        geo = dict(
            scope='europe',                                #Область карты 
            landcolor = "green",                           #Цвет для стран
            countrycolor = "black",                        #Граница стран
        ),
    )

In [ ]:
#Сохраняем данные в json
df_coord_border[['lon', 'lat']].to_json('data//border.json')